<h1>NFW internal Halo Profiles</h1>
This notebook contains code to output the halo profiles of the given mass and redshift at the given scale in fourier space.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

<h2>Setting up WMAP1 Cosmology parameters below</h2>
Source: <a href='https://arxiv.org/pdf/astro-ph/0302209.pdf'>arXiv paper</a> on <i>First Year Wilkinson Microwave Anisotropy Probe (WMAP) Observations:Determination of Cosmological Parameters</i> by Spergel et. al.

In [2]:
z=0.0   # Redshift
del_crit=1.69/(1.0+z)   # Critical overdensity for spherical collapse at given redshift

omega_m=0.270062   # Density parameter for matter in the Universe WMAP1 Cosmology (Niladri's=0.276)
h=0.72  # 100h km /s MPc is Hubble constant today WMAP1 Cosmology (Niladri's=0.7)

M_sun=1.989e+30  # M_sun in kg
Mega_parsec=3.086e+22  # parsec in metres

rho_cr=((3*np.power((100*h),2))/(8*np.pi*6.673e-11))*(Mega_parsec*1e+6*h/(M_sun))
# critical density of the Universe today 3H^2/8Pi*G in units M_sun.h^-1/MPc^3
rho_m=omega_m*rho_cr*((1+z)**3)  # where a=1/(1+z)

eps=1e-5   # To avoid divide by 0 and to set a cutoff limit for negligibly low values
finesse=1000   # Defines the granularity in calculating Si and Ci integrals

In [3]:
# This cell holds code to compute the variance for a given smoothening scale R defined by the mass passed as argument
PS=np.loadtxt("./../Normalize PS/NormalizedDimensionlessPS.txt")
k=PS[:,0]
ps=PS[:,1]  # ps=k^3*P/2*pi^2

const1=np.cbrt(3.0/(4*np.pi*rho_m))

# Smoothing window functions Fourier Transformed in k-space
def TopHat(k,R):
    return (3/np.power(k*R,3))*(np.sin(k*R)-(k*R)*np.cos(k*R))

def variance(m):
    R=const1*np.cbrt(m)
    function=ps*(TopHat(k,R)**2)/k
    result=np.trapz(function,k)
    return result

In [4]:
# The characteristic mass scale at which Nu(m,z)=del_crit**2/variance(Mstar)**2=1
# Goal is to find the mass for which when the variance(sigma^2) is calculated, it is equal to del_crit^2
def Mstar():
    Mlower=1e+10
    Mupper=1e+16
    Mst=-1
    while True:
        Mcheck=(Mupper+Mlower)/2.0
        x=variance(Mcheck)-(del_crit**2)
        if(0.0<=x<=eps):
            Mst=Mcheck
            break
        if(x<0.0):
            Mupper=Mcheck
        if(x>eps):
            Mlower=Mcheck
    
    print(Mst)
    return Mst

# Return the density at given scale k and mass of halo m for spherically symmetric profile truncated at virial radius
# reference page 34 in Cooray & Sheth


def Si(x):
    t=np.linspace(0,x,1000)+eps
    function=np.sin(t)/t
    return np.trapz(function,t)

# Possible fault regarding consideration of values of x
def Ci(x):
    limit=1/eps
    t=np.linspace(x,limit+x,limit)
#    t=np.linspace(x,limit*x,limit)
    function=np.cos(t)/t
    return -1.0*np.trapz(function,t)

# need: rho_s,r_s
def u_NFW(k,m):
    c=(9/(1+z))*((m/Mstar())**-0.13)
    
    util1=k*r_s
    util2=(1+c)*util1
    
    term1=4*np.pi*rho_s*r_s/m
    term2=np.sin(util1)*(Si(util2)-Si(util1))
    term3=-1.0*np.sin(c*util1)/util2
    term4=np.cos(util1)*(Ci(util2)-Ci(util1))
    
    result=term1*(term2+term3+term4)
    return result

Mstar()

51879205994.77365


51879205994.77365

In [ ]:
NFW_alpha=1.0
NFW_beta=2.0

# Returns density at given radius r and mass of halo m, from NFW Profile in Real Space - Rho(r|m)
def NFW(r,m):
    c_mean=(9/(1+z))*((m/m_star(z))**-0.13)
    
    term1=(r/r_s)
    term2=(1+term1)**NFW_beta
    term1=term1**NFW_alpha
    result=rho_s/(term1*term2)
    return result

In [ ]:
# Return the density at given scale k and mass of halo m for spherically symmetric profile truncated at virial radius
# reference page 34 in Cooray & Sheth

eps=1e-5   # To avoid divide by 0 and to set a cutoff limit for negligibly low values
finesse=1000   # Defines the number of

def Si(x):
    t=np.linspace(0,x,1000)+eps
    function=np.sin(t)/t
    return np.trapz(function,t)

# Possible fault regarding consideration of values of x
def Ci(x):
    limit=1/eps
    t=np.linspace(x,limit+x,limit)
#    t=np.linspace(x,limit*x,limit)
    function=np.cos(t)/t
    return -1.0*np.trapz(function,t)

# need: rho_s,r_s,m_star(z)
def u_NFW(k,m):
    c=(9/(1+z))*((m/m_star(z))**-0.13)
    
    util1=k*r_s
    util2=(1+c)*util1
    
    term1=4*np.pi*rho_s*r_s/m
    term2=np.sin(util1)*(Si(util2)-Si(util1))
    term3=-1.0*np.sin(c*util1)/util2
    term4=np.cos(util1)*(Ci(util2)-Ci(util1))
    
    result=term1*(term2+term3+term4)
    return result